# simple t-test

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pylab as plt
import matplotlib
from IPython.display import display, HTML, Image

import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests

%matplotlib inline

Load test data

In [4]:
count_data = pd.read_pickle('../static/data/debugging/count_data.df')
count_data.head()

,HK1,HK2,HK3,HK4,INFEC_1,INFEC_2,INFEC_3,INFEC_4,UN4,UN_1
compound_pk,,,,,,,,,,
C00009,236607.39,536029.50,321735.00,380636.19,479578.50,230005.63,232082.92,449130.78,312837.72,381401.81
C00020,87448.20,78115.14,79525.53,97963.03,66545.36,186587.56,129466.49,128112.98,135613.81,83745.11
C00025,26416422.00,20081542.00,22863020.00,20091068.00,21756356.00,17123612.00,37839940.00,19147462.00,18792648.00,20977542.00
C00037,1084838.38,766316.81,994945.63,830652.13,759096.00,854259.56,1467937.75,662653.81,1048188.25,1340801.63
C00041,3134556.25,2539419.50,2901517.75,2443864.00,4275319.00,3713087.00,7178702.00,3782108.25,2808829.75,3537903.00


In [5]:
col_data = pd.read_pickle('../static/data/debugging/col_data.df')
col_data

,group
sample,
UN_1,UN
UN4,UN
INFEC_1,INFEC
INFEC_2,INFEC
INFEC_3,INFEC
INFEC_4,INFEC
HK1,HK
HK2,HK
HK3,HK


Run t-test

In [6]:
case = 'INFEC'
control = 'UN'

In [7]:
sample_group = col_data[col_data['group'] == case]
case_data = count_data[sample_group.index]

In [8]:
sample_group = col_data[col_data['group'] == control]
control_data = count_data[sample_group.index]

In [9]:
nrow, _ = count_data.shape

In [10]:
pvalues = []
lfcs = []
for i in range(nrow):
    case_df = case_data.iloc[i, :]
    control_df = control_data.iloc[i, :]
    statistics, pvalue = stats.ttest_ind(np.log(case_df), np.log(control_df))

    case_avg = np.log(np.mean(case_df.values))
    control_avg = np.log(np.mean(control_df.values))
    lfc = case_avg - control_avg
    
    pvalues.append(pvalue)    
    lfcs.append(lfc)

In [11]:
reject, pvals_corrected, _, _ = multipletests(pvalues, method='fdr_bh')

In [12]:
pvals_corrected

array([8.71044849e-01, 8.00009824e-01, 7.78123277e-01, 7.75449024e-01,
       7.75449024e-01, 7.75449024e-01, 7.75449024e-01, 7.78123277e-01,
       7.75449024e-01, 7.75449024e-01, 7.75449024e-01, 7.75449024e-01,
       7.78123277e-01, 7.78123277e-01, 7.75449024e-01, 7.75449024e-01,
       7.78123277e-01, 7.75449024e-01, 7.75449024e-01, 5.61786486e-04,
       7.78123277e-01, 7.75449024e-01, 8.00009824e-01, 7.75449024e-01,
       7.75449024e-01, 8.00009824e-01])

In [13]:
lfcs

[0.0016686142689490424,
 0.15195012122953067,
 0.1867008564918642,
 -0.24387737822862476,
 0.40067523834332164,
 0.271329692187539,
 -0.3086017554856504,
 0.30743658921103467,
 -0.226168941979477,
 -0.22095074815854332,
 0.40067523834332164,
 -0.20850005901884217,
 -0.13547231259314252,
 -0.08439614917962857,
 -0.25070930914578504,
 -1.695324542123247,
 -0.18158896028329963,
 0.30704561535806363,
 -0.3349304194451985,
 1.1509768447585937,
 -0.1445827746708428,
 -0.30115347157380334,
 -0.3827334703347809,
 -0.22251773918966578,
 -0.2497255361997155,
 -0.047478874786788694]

In [14]:
result_df = pd.DataFrame({
    'padj': pvals_corrected,
    'log2FoldChange': lfcs
}, index=count_data.index)

In [15]:
result_df

,padj,log2FoldChange
compound_pk,,
C00009,0.871045,0.001669
C00020,0.800010,0.151950
C00025,0.778123,0.186701
C00037,0.775449,-0.243877
C00041,0.775449,0.400675
C00051,0.775449,0.271330
C00064,0.775449,-0.308602
C00077,0.778123,0.307437
C00078,0.775449,-0.226169
